<a href="https://colab.research.google.com/github/rodrihgh/MerryXmasEU/blob/main/Wikimedia_Image_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lxml aiohttp asyncio nest_asyncio aiofiles

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 271 kB 58.4 MB/s 
     |████████████████████████████████| 192 kB 48.2 MB/s 
     |████████████████████████████████| 160 kB 65.2 MB/s 


In [4]:
import shutil
from lxml import etree
from lxml import html
import aiohttp
import asyncio
import aiofiles
import nest_asyncio
import os

#fix for running in colab
nest_asyncio.apply()

#all categories will be compressed into storeDirectory + download.zip on completion

#ONLY CHNAGE THIS
url = 'https://commons.wikimedia.org/wiki/Category:Nativity_of_Jesus_Christ'
storeDirectory = '/content/'
checkForCategories = False

file_list = []

#DON'T CHANGE
tasks = []
categories = 0
categoryTasks = []
checkedCategories = []
completed = -1
totalImages = 0
completedImages = 0

async def fetch_page(session, url, cat = ''):
  try:
    async with session.get(url) as resp:
      source = await resp.text()

      dom = html.fromstring(source)

      return [cat, dom]
  except asyncio.TimeoutEraror or aiohttp.ClientConnectorError:
    #print('Timeout')
    return False

async def fetch_images(session, url):
  global totalImages

  dom = await fetch_page(session, url)

  #timeout error
  if dom == False:
    return

  images = dom[1].xpath('*//div[@class="thumb"]//a')
  subcategories = dom[1].xpath('*//div[@class="CategoryTreeItem"]//a')

  if(len(subcategories) > 0 and checkForCategories):
    for category in subcategories:
      if(category not in checkedCategories):
        categoryTasks.append(asyncio.ensure_future(fetch_images(session, 'https://commons.wikimedia.org' + category.attrib['href'])))
        checkedCategories.append(category)
        print('Found category', category.attrib['href'])

  if (len(images) > 0):
    totalImages += len(images)
    print("Found", len(images), "images")
    #download images for each category
    for image in images:
      cat = url.split('Category:')[1]
      tasks.append(asyncio.ensure_future(fetch_page(session, 'https://commons.wikimedia.org' + image.attrib['href'], cat)))

  global completed
  completed += 1

async def main(loop):
  global url
  global completedImages
  global file_list

  async with aiohttp.ClientSession(loop=loop) as session:
    await fetch_images(session, url)

    #fix to resolve finding all categories first
    while True:
      await asyncio.gather(*categoryTasks)

      #check if images have been found on all category pages
      if(completed == len(categoryTasks)):
        break

    pages = await asyncio.gather(*tasks)
    for page in pages:
      #timeout error
      if(page == False):
        continue

      cat = page[0]
      source = page[1]
       
      #print(cat, source.xpath('*//div[@class="fullImageLink"]//img')[0].attrib['src'])
      imgURL = source.xpath('*//div[@class="fullImageLink"]//img')[0].attrib['src']

      filename = imgURL.split('/')[-1]
      #TODO: save images into category folders
      async with session.get(imgURL) as resp:
        if resp.status == 200:
            if(os.path.isdir(storeDirectory + cat + '/') == False):
              os.mkdir(storeDirectory + cat + '/')
            fname = storeDirectory + cat + '/' + filename
            try:
              f = await aiofiles.open(fname, mode='wb')
              await f.write(await resp.read())
              await f.close()
              completedImages += 1
              file_list.append(imgURL)
              print(completedImages, '/', totalImages)
            except OSError:
              pass

  #create zip file to download
  shutil.make_archive(storeDirectory + 'download', 'zip', storeDirectory)

#main event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

print(file_list)

Found 94 images
1 / 94
2 / 94
3 / 94
4 / 94
5 / 94
6 / 94
7 / 94
8 / 94
9 / 94
10 / 94
11 / 94
12 / 94
13 / 94
14 / 94
15 / 94
16 / 94
17 / 94
18 / 94
19 / 94
20 / 94
21 / 94
22 / 94
23 / 94
24 / 94
25 / 94
26 / 94
27 / 94
28 / 94
29 / 94
30 / 94
31 / 94
32 / 94
33 / 94
34 / 94
35 / 94
36 / 94
37 / 94
38 / 94
39 / 94
40 / 94
41 / 94
42 / 94
43 / 94
44 / 94
45 / 94
46 / 94
47 / 94
48 / 94
49 / 94
50 / 94
51 / 94
52 / 94
53 / 94
54 / 94
55 / 94
56 / 94
57 / 94
58 / 94
59 / 94
60 / 94
61 / 94
62 / 94
63 / 94
64 / 94
65 / 94
66 / 94
67 / 94
68 / 94
69 / 94
70 / 94
71 / 94
72 / 94
73 / 94
74 / 94
75 / 94
76 / 94
77 / 94
78 / 94
79 / 94
80 / 94
81 / 94
82 / 94
83 / 94
84 / 94
85 / 94
86 / 94
87 / 94
88 / 94
89 / 94
90 / 94
91 / 94
92 / 94
93 / 94
['https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Weihnachten.jpg/800px-Weihnachten.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/27310_Oudenaarde_Sint-Walburgakerk_42.jpg/400px-27310_Oudenaarde_Sint-Walburgakerk_42.jpg'

In [1]:
from google.colab import drive
drive.flush_and_unmount()
